rPPG or Remote Photoplethysmography is a technique that extracts heart rate information from video data, and it can be used for liveness detection by analyzing the variations in the rPPG signal.


In [1]:
import cv2
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA


In [1]:
!pip3 install opencv-python-headless numpy scipy scikit-learn matplotlib mediapipe


Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Use the webcam as the video source
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")


In [3]:
def extract_roi(frame, face_cascade):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        # Get the first detected face
        x, y, w, h = faces[0]
        roi = frame[y:y + h, x:x + w]
        return roi
    else:
        return None

def extract_rppg_signal(roi):
    # Extract the green channel
    green_channel = roi[:, :, 1]

    # Compute the mean intensity along the time axis
    rppg_signal = np.mean(green_channel, axis=1)

    return rppg_signal

def estimate_heart_rate(rppg_signal, fps):
    # Perform FastICA to remove motion artifacts
    ica = FastICA(n_components=1)
    rppg_cleaned = ica.fit_transform(rppg_signal.reshape(-1, 1))

    # Find peaks in the rPPG signal
    peaks, _ = find_peaks(rppg_cleaned[:, 0], height=0.5)

    # Calculate heart rate in beats per minute (BPM)
    heart_rate = len(peaks) / (len(rppg_cleaned) / fps) * 60

    return heart_rate


In [ ]:
# Create a cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Set the frames per second (fps)
fps = 30

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break

    # Extract the ROI (face) from the frame
    roi = extract_roi(frame, face_cascade)

    if roi is not None:
        # Extract the rPPG signal from the ROI
        rppg_signal = extract_rppg_signal(roi)

        # Estimate the heart rate
        heart_rate = estimate_heart_rate(rppg_signal, fps)

        # Display the heart rate on the frame
        cv2.putText(frame, f"Heart Rate: {heart_rate:.1f} BPM", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with heart rate information
    cv2.imshow("rPPG Liveness Detection", frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


2024-12-04 09:44:28.150 Python[60018:17714058] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


: 